In [3]:
import torch
from torch import nn
from torch.ao import quantization
from torchvision import models
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


class QuantizedVGG16(nn.Module):
    def __init__(self, model_fp32):
        super(QuantizedVGG16, self).__init__()
        self.quant = quantization.QuantStub()
        self.dequant = quantization.DeQuantStub()
        self.model_fp32 = model_fp32
    
    def forward(self, x):
        x = self.quant(x)
        x = self.model_fp32(x)
        x = self.dequant(x)
        return x

hyperparams = {
    "batch_size": 4,
    "learning_rate": 0.0001,
    "epochs": 5,
    "transform": transforms.Compose(
        [
            transforms.Resize(128),
            transforms.CenterCrop(128),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.48235, 0.45882, 0.40784],
                std=[0.229, 0.224, 0.225],
            ),
        ]
    ),
}

model = models.vgg16(num_classes=2)
model.load_state_dict(torch.load("../models/VGG16.pt"))
#model.classifier[6] = torch.nn.Linear(4096, 2)
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else "cpu"

#torch.backends.quantized.engine = 'qnnpack'
#device = torch.device("cpu")
quantized_model = QuantizedVGG16(model).to(device)

quantization_backend = "fbgemm"
quantized_model.qconfig = quantization.get_default_qconfig(quantization_backend)

model_static_quantized = quantization.prepare(quantized_model)

calibration_dataset = ImageFolder(
    "../datasets/pet/test",
    transform=hyperparams["transform"]
)
calibration_dataloader = DataLoader(
    calibration_dataset,
    batch_size=hyperparams["batch_size"]
)

model_static_quantized.to("cpu")

for i, (images, target) in enumerate(calibration_dataloader):
    if i >= 10:
        break
    model_static_quantized(images.to(device))

model_static_quantized.to("cpu")
model_static_quantized = quantization.convert(model_static_quantized)

torch.jit.save(torch.jit.script(model_static_quantized), "../models/PTSQ_VGG16.pt")

/Users/chohi/project/ai/DeepLearningDeepen/DeepLearningAdv/.venv/lib/python3.9/site-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


RuntimeError: Didn't find engine for operation quantized::conv2d_prepack NoQEngine

In [2]:
import torch
import torchvision.models as models

# 사전 훈련된 VGG16 모델 로드
#vgg16 = models.vgg16(pretrained=True)

vgg16 = models.vgg16(num_classes=2)


device = "cuda" if torch.cuda.is_available() else "cpu"

#model.classifier[6] = torch.nn.Linear(4096, 2)
# 평가 모드로 설정
vgg16.eval()

# 모델을 'VGG16.pt'로 저장
#torch.save(vgg16.state_dict(), 'VGG16.pt')
torch.save(vgg16.state_dict(), "../models/VGG16.pt")


print("VGG16 모델이 VGG16.pt로 저장되었습니다.")


VGG16 모델이 VGG16.pt로 저장되었습니다.
